# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from PositionalModel import PositionalModel

from src.match_names import *
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "Avg_FPL_points", "xP_4", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(FEATURES_OUTFIELD)

47

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape

(34514, 274)

In [7]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [8]:
# df = df[df["GW"] == 2]

In [9]:
# X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]
df = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [10]:
df.isna().sum().sort_values(ascending=False).head(30)

Min_points_2     16744
Min_points       16744
FPL_GW           16744
Clean_name       16744
Min_points_30    16744
Min_points_4     16744
Cmp%             15960
Carries          15954
PrgP             15954
Blocks           15954
Att              15954
PrgC             15954
Cmp              15954
GCA              15954
SCA              15954
xA               15954
T_att            15954
npxG             15954
T_succ           15954
xG               15954
Touches          15954
Tkl              15954
xP               15915
Min              15841
FPL_ast_30       15840
Bonus_30         15840
Att_4            15840
Name_4           15840
Creativity_30    15840
Player_CS_30     15840
dtype: int64

In [11]:
# df[FEATURES_OUTFIELD].iloc[0]

In [12]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [13]:
df.shape

(16744, 274)

In [14]:
X_test = df.copy()

In [15]:
file = open("models/GBR_positional_XGB.pkl",'rb')
model = pickle.load(file)

In [16]:
model

In [17]:
model.predict( X_test.head(5) )

[1.965268870021986,
 2.1307998496553173,
 1.9335245643892598,
 1.6818710719385457,
 2.2573340092935874]

In [18]:
model.predict( X_test.head(3) )

[1.965268870021986, 2.1307998496553173, 1.9335245643892598]

In [19]:
model.predict( X_test.head(4) )

[1.965268870021986, 2.1307998496553173, 1.9335245643892598, 1.6818710719385457]

In [20]:
X_test.head(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
107,107,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117318,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True
108,108,2023-08-26,Sat,Home,Brentford,Crystal Palace,Aaron-Hickey,1.0,RB,90.0,...,0.69,1774.125598,1769.467049,NaN,1.466667,1.0,0.0,0.0,0.0,True


In [21]:
X_test.head(3)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True


# Predicting

In [22]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [23]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [24]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [25]:
X_test[FEATURES_OUTFIELD].isna()

,Was_home,Rating_difference,Price,Transfers_balance,Avg_FPL_points,Min_2,Gls_2,Sh_2,SoT_2,Ast_2,...,Ast_30,xG_30,xA_30,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,Player_GC_30,FPL_points_30,xP_30
76,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
77,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
107,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
108,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34509,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34510,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34511,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34512,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
# X_test.shape,  model.predict(X_test).shape

In [27]:
%%time
preds = get_predictions(model, df, X_test)

16744 (16744, 274)
CPU times: total: 2.69 s
Wall time: 1.29 s


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [28]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Avg_FPL_points,xP_4,FPL_pos,Price,Pred
12105,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,6.828947,6.325,MID,12.5,6.643354
6316,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,6.333333,6.000,GK,5.0,5.716798
5514,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,7.461538,4.950,FWD,14.0,5.560019
15802,Trent Alexander-Arnold,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,4.906667,4.050,DEF,7.9,5.143104
13363,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,4.275362,6.400,MID,7.6,4.626968
7784,Jarrod Bowen,4.0,2023-24,West Ham,Luton,0.0,1787.825562,1602.477417,4.857143,5.750,MID,7.0,4.467700
1498,Alisson Ramses Becker,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,4.565789,4.000,GK,5.5,4.288741
1008,Aaron Ramsdale,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,3.727273,4.000,GK,5.0,4.271567
1428,Alexis Mac Allister,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,3.078947,1.775,MID,6.0,4.226788
2930,Bruno Borges Fernandes,4.0,2023-24,Manchester Utd,Arsenal,0.0,1865.755249,1927.032104,4.800000,8.100,MID,8.5,4.211280


In [29]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Avg_FPL_points,xP_4,FPL_pos,Price,Pred
6316,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,6.333333,6.000000e+00,GK,5.0,5.716798
1498,Alisson Ramses Becker,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,4.565789,4.000000e+00,GK,5.5,4.288741
1008,Aaron Ramsdale,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,3.727273,4.000000e+00,GK,5.0,4.271567
5026,Ederson Santana de Moraes,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,3.717949,3.750000e+00,GK,5.5,3.833530
8937,José Malheiro de Sá,4.0,2023-24,Wolves,Crystal Palace,0.0,1704.537109,1758.725586,3.898734,1.000000e+00,GK,5.0,3.786916
1918,André Onana,4.0,2023-24,Manchester Utd,Arsenal,0.0,1865.755249,1927.032104,3.666667,5.500000e+00,GK,5.0,3.596771
13989,Robert Sánchez,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,3.333333,2.000000e+00,GK,4.5,3.428437
12628,Nick Pope,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,3.868421,3.500000e+00,GK,5.5,3.329913
5374,Emiliano Martínez Romero,4.0,2023-24,Aston Villa,Liverpool,0.0,1825.741455,1946.521606,3.692308,2.000000e+00,GK,5.0,3.179193
12733,Norberto Murara Neto,4.0,2023-24,Bournemouth,Brentford,0.0,1660.290894,1835.573242,3.242424,4.250000e+00,GK,4.5,2.754305


In [30]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Avg_FPL_points,xP_4,FPL_pos,Price,Pred
12127,Mohamed Salah,26.0,2023-24,Liverpool,Luton,1.0,1946.521606,1602.477417,6.828947,6.325,MID,12.5,6.958318
12132,Mohamed Salah,31.0,2023-24,Liverpool,Sheffield United,1.0,1946.521606,1631.764282,6.828947,6.325,MID,12.5,6.738356
12125,Mohamed Salah,24.0,2023-24,Liverpool,Burnley,1.0,1946.521606,1722.952881,6.828947,6.325,MID,12.5,6.674173
12139,Mohamed Salah,38.0,2023-24,Liverpool,Wolves,1.0,1946.521606,1704.537109,6.828947,6.325,MID,12.5,6.671228
12110,Mohamed Salah,9.0,2023-24,Liverpool,Everton,1.0,1946.521606,1691.463989,6.828947,6.325,MID,12.5,6.671228
12107,Mohamed Salah,6.0,2023-24,Liverpool,West Ham,1.0,1946.521606,1787.825562,6.828947,6.325,MID,12.5,6.666308
12123,Mohamed Salah,22.0,2023-24,Liverpool,Chelsea,1.0,1946.521606,1781.043579,6.828947,6.325,MID,12.5,6.666308
12115,Mohamed Salah,14.0,2023-24,Liverpool,Fulham,1.0,1946.521606,1732.639404,6.828947,6.325,MID,12.5,6.666308
12134,Mohamed Salah,33.0,2023-24,Liverpool,Crystal Palace,1.0,1946.521606,1758.725586,6.828947,6.325,MID,12.5,6.666308
12113,Mohamed Salah,12.0,2023-24,Liverpool,Brentford,1.0,1946.521606,1835.573242,6.828947,6.325,MID,12.5,6.643354


In [31]:
preds[preds["FPL_pos"] == "GK"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Avg_FPL_points,xP_4,FPL_pos,Price,Pred
315,Guglielmo Vicario,2.0,2023-24,Tottenham,Manchester Utd,1.0,1817.656738,1877.037720,6.333333,1.0,GK,5.0,6.324765
314,Guglielmo Vicario,1.0,2023-24,Tottenham,Brentford,0.0,1815.873413,1824.330444,6.333333,1.0,GK,5.0,6.022122
6349,Guglielmo Vicario,37.0,2023-24,Tottenham,Burnley,1.0,1829.239014,1722.952881,6.333333,6.0,GK,5.0,5.735928
6316,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,6.333333,6.0,GK,5.0,5.716798
6330,Guglielmo Vicario,18.0,2023-24,Tottenham,Everton,1.0,1829.239014,1691.463989,6.333333,6.0,GK,5.0,5.706786
6337,Guglielmo Vicario,25.0,2023-24,Tottenham,Wolves,1.0,1829.239014,1704.537109,6.333333,6.0,GK,5.0,5.706786
6339,Guglielmo Vicario,27.0,2023-24,Tottenham,Crystal Palace,1.0,1829.239014,1758.725586,6.333333,6.0,GK,5.0,5.705868
6335,Guglielmo Vicario,23.0,2023-24,Tottenham,Everton,0.0,1829.239014,1691.463989,6.333333,6.0,GK,5.0,5.687655
6324,Guglielmo Vicario,12.0,2023-24,Tottenham,Wolves,0.0,1829.239014,1704.537109,6.333333,6.0,GK,5.0,5.687655
6322,Guglielmo Vicario,10.0,2023-24,Tottenham,Crystal Palace,0.0,1829.239014,1758.725586,6.333333,6.0,GK,5.0,5.686738


In [32]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Mohamed Salah,226.575663
Guglielmo Vicario,207.873365
Erling Haaland,201.005161
Bruno Borges Fernandes,184.000105
Trent Alexander-Arnold,172.935069
Alisson Ramses Becker,159.889228
Phil Foden,158.465708
José Malheiro de Sá,158.078681
Miguel Almirón Rejala,156.651304


# Saving to file

In [33]:
preds.to_csv("predictions/preds_next_season.csv", index=False)